In [1]:
import sys 
sys.path.append('..')

In [2]:
import time
import pickle 
import numpy as np 
import pandas as pd
import networkx as nx 

from juliacall import Main as jl  
from hdna import *

In [3]:
# Import experimental data inside a dataframe

expdata = pd.read_csv('../data/herteldata.csv', names=['seq', 'expvalue'])

In [4]:
# Clean the dataframe 

expdata = expdata.drop(0)
expdata['expvalue'] = ['{:e}'.format(float(e)) for e in expdata['expvalue']]

In [5]:
# Actual computation 

rates = []

model = Model('dna', '3D')

for i, (seq, exp) in enumerate(zip(expdata['seq'], expdata['expvalue'])):
    seq = str(seq.strip())      # make sure they are alright
    exp = float(exp.strip())    #
    print(f'Creating network from string {seq}...')
    A = Strand(model, seq)
    B = A.complementary()
    kinet = Kinetwork(model, A, B, 1)
    geo = Geometry(120, 360)
    K = Kinetics(model, kinet, geo)
    opts = Options(method='direct', runtime=3e-6, Nmonte=20000)
    print('embedding strand into biosimulator network model...')
    simulatore = Simulator(model, kinet, K, options=opts)
    print('start running simulations...')
    results = simulatore.ensemble()
    mfpt = simulatore.mfpts(results)
    rates.append(1/mfpt)
    print(f"experimental rate: {'{:e}'.format(exp)}")
    print(f"computed rate:     {'{:e}'.format(1/mfpt)}")
    break 

Creating network from string GTTCGGTCTA...
embedding strand into biosimulator network model...
start running simulations...


100%|██████████| 20000/20000 [00:13<00:00, 1526.39it/s]


3043 simulations didn't produce a duplex.
That's 15.215% of simulations
experimental rate: 1.145950e+06
computed rate:     8.792824e+05


In [6]:
from pyvis.network import Network

In [8]:
G = simulatore.Graph
for n in G.nodes.data():
    del n[1]['object']

In [33]:
a = list(G.edges.data())[0][2]
[(b,a[b]) for b in [*a]]

[('kind', 'on_nucleation'),
 ('kf', 38505212.66779661),
 ('kb', 827964.3174347384)]

In [39]:
states = [node[1]['state'] for node in list(G.nodes.data())]
visnet = Network(select_menu=True,filter_menu=True, heading = 'stringseq')
visnet.add_nodes(list(G.nodes()), state=states)
for edge in list(G.edges.data()):
    visnet.add_edge(edge[0], edge[1], **edge[2])
visnet.show_buttons()
visnet.toggle_physics(True)
visnet.show('name.html')

In [11]:
states = [node[1]['state'] for node in list(G.nodes.data())]

visnet = Network(select_menu=True,filter_menu=True)
visnet.add_nodes(list(G.nodes()), state=states)

for edge in list(G.edges()):
    pass

visnet.add_edges(list(G.edges()))
visnet.show_buttons()
visnet.toggle_physics(True)
visnet.show('name.html')

In [17]:
visnet.nodes

[{'color': '#97c2fc',
  'state': 'singlestranded',
  'id': '..........+..........',
  'label': '..........+..........',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '.(........+.)........',
  'label': '.(........+.)........',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '..(.......+.)........',
  'label': '..(.......+.)........',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '((........+...)).....',
  'label': '((........+...)).....',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '(.((......+..)).)....',
  'label': '(.((......+..)).)....',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '......(...+.)........',
  'label': '......(...+.)........',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_register',
  'id': '.(..(.....+....)..)..',
  'label': '.(..(.....+....)..)..',
  'shape': 'dot'},
 {'color': '#97c2fc',
  'state': 'off_r

In [ ]:
G = simulatore.Graph
Gp = nx.convert_node_labels_to_integers(G)
Np = Network(notebook=True, cdn_resources='in_line')
Np.from_nx(Gp)
Np.show_buttons()
Np.show('Try.html')

In [ ]:
expdata['computed'] = rates
expdata['ratios'] = [float(e)/c for e, c in zip(expdata['expvalue'], expdata['computed'])]

In [ ]:
expdata.to_csv('nucleation1.csv')